### Домашнее задание

Переписать загрузку данных с python функций на Dataset и Dataloader и применить сеть с attention

In [1]:
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset

import numpy as np

In [2]:
batch_size = 64
epochs = 30
latent_dim = 256
num_samples = 10000
data_path = './data6/fra-eng/fra.txt'

In [3]:
# Собираем из текстов токены

input_texts = []
target_texts = []

input_vocab = set()
output_vocab = set()


with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    for word in input_text.split():
        input_vocab.add(word.strip())
    target_texts.append(target_text)
    for word in target_text.split():
        output_vocab.add(word.strip())
    
input_vocab2index = {word: i+2 for i, word in enumerate(input_vocab)}
output_vocab2index = {word: i+2 for i, word in enumerate(output_vocab)}

In [4]:
input_vocab = list(input_vocab)
output_vocab = list(output_vocab)

In [5]:
class Input(Dataset):
    def __init__(self):
        self.input_vocab = input_vocab

    def __len__(self):
        return len(self.input_vocab)

    def __getitem__(self, idx):
        return self.input_vocab[idx]
    
    def indexesFromSentence(sentence, vocab):
        return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

    def tensorFromSentence(sentence, vocab):
        indexes = indexesFromSentence(sentence, vocab)
        indexes.append(1)
        return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

    def tensorsFromSent(input_sentences):
        input_tensor = tensorFromSentence(input_sentences, input_vocab2index)
        return (input_tensor)

if __name__ == '__main__':
    dataset = Input()
    print(len(dataset))
    print(dataset[50])

3111
approves.


In [6]:
class Output(Dataset):
    def __init__(self):
        self.output_vocab = output_vocab

    def __len__(self):
        return len(self.output_vocab)

    def __getitem__(self, idx):
        return self.output_vocab[idx]
    
    def indexesFromSentence(sentence, vocab):
        return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

    def tensorFromSentence(sentence, vocab):
        indexes = indexesFromSentence(sentence, vocab)
        indexes.append(1)
        return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

    def tensorsFromSent(output_sentences):
        target_tensor = tensorFromSentence(input_sentences, output_vocab2index)
        return (target_tensor)

if __name__ == '__main__':
    dataset = Output()
    print(len(dataset))
    print(dataset[50])

5757
amuse-toi.


In [7]:
df_train, df_test = Input(), Output()

In [8]:
train_loader = torch.utils.data.DataLoader(df_train,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           num_workers=0)

test_loader = torch.utils.data.DataLoader(df_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          num_workers=0)

In [9]:
def indexesFromSentence(sentence, vocab):
    return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

def tensorFromSentence(sentence, vocab):
    indexes = indexesFromSentence(sentence, vocab)
    indexes.append(1)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)


def tensorsFromSent(input_sentences, output_sentences):
    input_tensor = tensorFromSentence(input_sentences, input_vocab2index)
    target_tensor = tensorFromSentence(output_sentences, output_vocab2index)
    return (input_tensor, target_tensor)

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [11]:
class DecoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_, hidden, encoder_outputs):
        embedded = self.embedding(input_).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden.view(1, 1, 1, -1))
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [13]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == 1:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
encoder = EncoderRNN(len(input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(output_vocab2index)+2, dropout_p=0.1)

#attn_decoder1 = DecoderRNN(len(output_vocab2index)+2, 30)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
training_pairs = np.random.randint(0, len(input_texts), size=10000)
criterion = nn.NLLLoss()

print_loss_total = 0
for i in range(1000):
    input_tensor, target_tensor = tensorsFromSent(input_texts[training_pairs[i]], target_texts[training_pairs[i]])

    loss = train(input_tensor, target_tensor, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print('(%d %d%%) %.4f' % (i, i / 10 * 100, print_loss_avg))

(0 0%) 8.5609
(1 10%) 8.7300
(2 20%) 8.6807
(3 30%) 8.6533
(4 40%) 8.7391
(5 50%) 8.4671
(6 60%) 8.6613
(7 70%) 8.5622
(8 80%) 8.5606
(9 90%) 8.5641
(10 100%) 8.6097
(11 110%) 8.6060
(12 120%) 8.6144
(13 130%) 8.5854
(14 140%) 8.5922
(15 150%) 8.5346
(16 160%) 8.6071
(17 170%) 4.3124
(18 180%) 8.5905
(19 190%) 8.5432
(20 200%) 6.5094
(21 210%) 2.4786
(22 220%) 8.5457
(23 229%) 8.5423
(24 240%) 8.5402
(25 250%) 4.3090
(26 260%) 8.4316
(27 270%) 8.5338
(28 280%) 3.4414
(29 290%) 1.7032
(30 300%) 4.2687
(31 310%) 4.4129
(32 320%) 3.4390
(33 330%) 3.3885
(34 340%) 6.8198
(35 350%) 2.8320
(36 360%) 4.2586
(37 370%) 2.8885
(38 380%) 5.1315
(39 390%) 4.3838
(40 400%) 4.2994
(41 409%) 8.6629
(42 420%) 2.0949
(43 430%) 3.4752
(44 440%) 2.8273
(45 450%) 2.8379
(46 459%) 4.1699
(47 470%) 4.3455
(48 480%) 5.9133
(49 490%) 5.8873
(50 500%) 4.3277
(51 509%) 5.0988
(52 520%) 5.7807
(53 530%) 2.4639
(54 540%) 5.0952
(55 550%) 5.1763
(56 560%) 1.4530
(57 570%) 6.5622
(58 580%) 4.2463
(59 590%) 8.4455
(

(450 4500%) 4.2823
(451 4510%) 4.2096
(452 4520%) 1.7249
(453 4530%) 2.7219
(454 4540%) 2.7679
(455 4550%) 3.4102
(456 4560%) 3.6603
(457 4570%) 3.2852
(458 4580%) 3.2610
(459 4590%) 2.2486
(460 4600%) 3.4327
(461 4610%) 3.4979
(462 4620%) 2.5449
(463 4630%) 5.7862
(464 4640%) 3.6546
(465 4650%) 2.5435
(466 4660%) 3.5109
(467 4670%) 3.5555
(468 4680%) 4.3631
(469 4690%) 3.7370
(470 4700%) 6.0405
(471 4710%) 3.9448
(472 4720%) 3.3454
(473 4730%) 4.2109
(474 4740%) 2.1497
(475 4750%) 3.3103
(476 4760%) 3.8052
(477 4770%) 3.5577
(478 4780%) 2.1163
(479 4790%) 4.3181
(480 4800%) 5.9763
(481 4810%) 4.4537
(482 4820%) 3.6978
(483 4830%) 3.7595
(484 4840%) 2.1653
(485 4850%) 5.8662
(486 4860%) 4.4530
(487 4870%) 4.3228
(488 4880%) 3.3643
(489 4890%) 3.4277
(490 4900%) 3.2527
(491 4910%) 1.9382
(492 4920%) 3.1412
(493 4930%) 2.6332
(494 4940%) 4.3660
(495 4950%) 2.2614
(496 4960%) 3.3500
(497 4970%) 3.6481
(498 4980%) 4.1458
(499 4990%) 3.3149
(500 5000%) 5.2700
(501 5010%) 2.3947
(502 5020%) 

(897 8970%) 3.5051
(898 8980%) 0.8193
(899 8990%) 3.3247
(900 9000%) 5.4609
(901 9010%) 1.9391
(902 9020%) 3.7431
(903 9030%) 6.4488
(904 9040%) 6.5875
(905 9050%) 5.7826
(906 9060%) 1.0259
(907 9070%) 6.9079
(908 9080%) 0.9773
(909 9090%) 5.1056
(910 9100%) 5.3012
(911 9110%) 5.4456
(912 9120%) 6.1552
(913 9130%) 5.0608
(914 9140%) 2.3290
(915 9150%) 2.6340
(916 9160%) 1.8932
(917 9170%) 3.2356
(918 9180%) 1.7427
(919 9190%) 0.8359
(920 9200%) 3.3007
(921 9210%) 4.9826
(922 9220%) 3.1634
(923 9230%) 6.1926
(924 9240%) 5.0756
(925 9250%) 1.7487
(926 9260%) 4.9537
(927 9270%) 4.8321
(928 9280%) 3.7545
(929 9290%) 4.3915
(930 9300%) 6.2651
(931 9310%) 3.1870
(932 9320%) 3.7547
(933 9330%) 2.3100
(934 9340%) 2.5958
(935 9350%) 4.5649
(936 9360%) 2.9295
(937 9370%) 6.3052
(938 9380%) 2.1842
(939 9390%) 3.8323
(940 9400%) 4.0369
(941 9410%) 2.2654
(942 9420%) 2.2144
(943 9430%) 2.4930
(944 9440%) 1.9472
(945 9450%) 3.5273
(946 9460%) 3.9638
(947 9470%) 6.0239
(948 9480%) 2.5853
(949 9490%) 